In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e12/sample_submission.csv
/kaggle/input/playground-series-s5e12/train.csv
/kaggle/input/playground-series-s5e12/test.csv


In [2]:
data = pd.read_csv('/kaggle/input/playground-series-s5e12/train.csv')
test = pd.read_csv("/kaggle/input/playground-series-s5e12/test.csv")

In [3]:
data.head()
data=data.set_index(data['id'])
data.drop(columns=['id'],axis=1,inplace=True)
test = test.set_index('id')


In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 700000 entries, 0 to 699999
Data columns (total 25 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   age                                 700000 non-null  int64  
 1   alcohol_consumption_per_week        700000 non-null  int64  
 2   physical_activity_minutes_per_week  700000 non-null  int64  
 3   diet_score                          700000 non-null  float64
 4   sleep_hours_per_day                 700000 non-null  float64
 5   screen_time_hours_per_day           700000 non-null  float64
 6   bmi                                 700000 non-null  float64
 7   waist_to_hip_ratio                  700000 non-null  float64
 8   systolic_bp                         700000 non-null  int64  
 9   diastolic_bp                        700000 non-null  int64  
 10  heart_rate                          700000 non-null  int64  
 11  cholesterol_total              

In [5]:
data['gender'] = data['gender'].astype('category')
data['ethnicity'] = data['ethnicity'].astype('category')
data['education_level'] = data['education_level'].astype('category')
data['income_level'] = data['income_level'].astype('category')
data['smoking_status'] = data['smoking_status'].astype('category')
data['employment_status'] = data['employment_status'].astype('category')



In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700000 entries, 0 to 699999
Data columns (total 25 columns):
 #   Column                              Non-Null Count   Dtype   
---  ------                              --------------   -----   
 0   age                                 700000 non-null  int64   
 1   alcohol_consumption_per_week        700000 non-null  int64   
 2   physical_activity_minutes_per_week  700000 non-null  int64   
 3   diet_score                          700000 non-null  float64 
 4   sleep_hours_per_day                 700000 non-null  float64 
 5   screen_time_hours_per_day           700000 non-null  float64 
 6   bmi                                 700000 non-null  float64 
 7   waist_to_hip_ratio                  700000 non-null  float64 
 8   systolic_bp                         700000 non-null  int64   
 9   diastolic_bp                        700000 non-null  int64   
 10  heart_rate                          700000 non-null  int64   
 11  cholesterol_total 

In [7]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [8]:
X = data.drop(columns='diagnosed_diabetes')
y = data['diagnosed_diabetes'].copy()
y.sum()/len(y)

0.6232957142857143

In [9]:
X_train ,X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , random_state=42 , stratify=y)
y_train.sum()/len(y_train),y_test.sum()/len(y_test)
ratio = (len(y_train) - y_train.sum()) / y_train.sum()
ratio

0.6043730625762793

In [10]:
mode = xgb.XGBClassifier(tree_method='hist',
                          enable_categorical=True,
                        objective='binary:logistic',gamma =0 ,early_stopping_rounds = 200,n_estimators=10000, learning_rate=0.01 , max_depth=6 , reg_lambda=2,scale_pos_weight=ratio
                        )
mode.fit(X_train , y_train , verbose=True , eval_metric='auc' , eval_set=[(X_test,y_test)])
params={'max_depth' : [4,5,6,7,8],'learning_rate':[0.1,1],'gamma':[0,1,2],'reg_lambda':[1,2,3]}

/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-auc:0.68581
[1]	validation_0-auc:0.68711
[2]	validation_0-auc:0.68703
[3]	validation_0-auc:0.68719
[4]	validation_0-auc:0.68771
[5]	validation_0-auc:0.68769
[6]	validation_0-auc:0.68798
[7]	validation_0-auc:0.68847
[8]	validation_0-auc:0.68868
[9]	validation_0-auc:0.68898
[10]	validation_0-auc:0.68888
[11]	validation_0-auc:0.68907
[12]	validation_0-auc:0.68913
[13]	validation_0-auc:0.68944
[14]	validation_0-auc:0.68971
[15]	validation_0-auc:0.68993
[16]	validation_0-auc:0.69009
[17]	validation_0-auc:0.69016
[18]	validation_0-auc:0.69029
[19]	validation_0-auc:0.69037
[20]	validation_0-auc:0.69046
[21]	validation_0-auc:0.69051
[22]	validation_0-auc:0.69080
[23]	validation_0-auc:0.69095
[24]	validation_0-auc:0.69110
[25]	validation_0-auc:0.69129
[26]	validation_0-auc:0.69138
[27]	validation_0-auc:0.69149
[28]	validation_0-auc:0.69158
[29]	validation_0-auc:0.69168
[30]	validation_0-auc:0.69177
[31]	validation_0-auc:0.69185
[32]	validation_0-auc:0.69189
[33]	validation_0-au

In [11]:
# grids = GridSearchCV(estimator=mode,param_grid=params ,scoring='roc_auc',verbose=2,n_jobs=-1 , cv=3)

In [12]:
# grids.fit(X_train,y_train)

In [13]:
# grids.best_params_
proba = mode.predict_proba(X_test)[:,1]

In [14]:
from sklearn.metrics import roc_auc_score

In [15]:
# roc_auc_score (grids.best_estimator_.predict_proba(X_test),y_test)
roc_auc_score(y_test,proba)

0.7254593904032905

In [16]:
cat_features = [
    "gender", "ethnicity", "education_level",
    "income_level", "smoking_status", "employment_status"
]

In [17]:
from catboost import CatBoostClassifier , Pool

In [18]:
# model = CatBoostClassifier(iterations=3000,learning_rate =0.03 , depth = 5  ,l2_leaf_reg=3, loss_function='Logloss',eval_metric='AUC',bootstrap_type="Bayesian",random_seed=42,
#     od_type="Iter",             
#     od_wait=100,)
train_pool = Pool(X_train , y_train , cat_features = cat_features)
valid_pool = Pool(X_test , y_test , cat_features = cat_features)
# model.fit(train_pool , eval_set=valid_pool , verbose=1, use_best_model=True)

In [19]:
test
test['gender'] = test['gender'].astype('category')
test['ethnicity'] = test['ethnicity'].astype('category')
test['education_level'] = test['education_level'].astype('category')
test['income_level'] = test['income_level'].astype('category')
test['smoking_status'] = test['smoking_status'].astype('category')
test['employment_status'] = test['employment_status'].astype('category')

In [20]:
# sub_pool = Pool(test,cat_features =cat_features)
# proba_cat = model.predict_proba(X_test)

In [21]:
# roc_auc_score (y_test, proba_cat[:,1])

In [22]:
test
test['gender'] = test['gender'].astype('category')
test['ethnicity'] = test['ethnicity'].astype('category')
test['education_level'] = test['education_level'].astype('category')
test['income_level'] = test['income_level'].astype('category')
test['smoking_status'] = test['smoking_status'].astype('category')
test['employment_status'] = test['employment_status'].astype('category')


In [23]:
sub_pred = mode.predict_proba(test)[:,1]
sub_pred


array([0.36165434, 0.5389241 , 0.67612076, ..., 0.49203497, 0.4737803 ,
       0.4929258 ], dtype=float32)

In [24]:
proba = mode.predict_proba(test)[:,1]
submission = pd.DataFrame({
    "id": test.index,
    "diagnosed_diabetes": proba
})

In [25]:
submission.to_csv("XGBoost.csv", index=False)

In [26]:
# train_set = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
# valid_set = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_features)


In [27]:
params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",

    "learning_rate": 0.01,
    "num_leaves": 31,
    "max_depth": -1,

    "feature_fraction": 0.8,   
    "bagging_fraction": 0.8,  
    "bagging_freq": 5,

    "min_data_in_leaf": 50,
    "lambda_l1": 2.0,
    "lambda_l2": 2.0,

    "max_bin": 255,
    "verbosity": 500,
    "force_col_wise": True
}


In [28]:
# from lightgbm import early_stopping, log_evaluation

# model = lgb.train(
#     params,
#     train_set,
#     num_boost_round=8000,
#     valid_sets=[train_set, valid_set],
#     valid_names=["train", "valid"],
#     callbacks=[
#         early_stopping(stopping_rounds=300),
#         log_evaluation(period=50)
#     ]
# )

In [29]:
# sub_pred = model.predict(test)
# sub_pred


In [30]:
# submission = pd.DataFrame({
#     "id": test.index,
#     "diagnosed_diabetes": sub_pred
# })

In [31]:
# submission.to_csv("submission_catboost.csv", index=False)

In [32]:
param_grid = {
    "depth": [4, 5, 6, 7, 8, 9, 10],
    "learning_rate": [0.005, 0.01, 0.02, 0.03],
    "l2_leaf_reg": [1, 3, 5, 7, 10, 15, 20],
    "bagging_temperature": [0.1, 0.5, 1.0, 2.0, 5.0],
    "random_strength": [0.5, 1, 1.5, 2, 3],
    "border_count": [32, 64, 128, 254],
    "grow_policy": ["SymmetricTree", "Lossguide"]
}


In [33]:
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV


model = CatBoostClassifier(
    iterations=3000,
   
    eval_metric="AUC",
    verbose=1,
    custom_loss=['AUC']
)

train_pool = Pool(X_train , y_train , cat_features = cat_features)
valid_pool = Pool(X_test , y_test , cat_features = cat_features)
model.fit(train_pool , eval_set=valid_pool , verbose=1, use_best_model=True,plot=True)







MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.093792
0:	test: 0.6787235	best: 0.6787235 (0)	total: 699ms	remaining: 34m 55s
1:	test: 0.6861362	best: 0.6861362 (1)	total: 1.26s	remaining: 31m 31s
2:	test: 0.6896205	best: 0.6896205 (2)	total: 1.72s	remaining: 28m 36s
3:	test: 0.6899109	best: 0.6899109 (3)	total: 2.28s	remaining: 28m 25s
4:	test: 0.6910322	best: 0.6910322 (4)	total: 2.78s	remaining: 27m 46s
5:	test: 0.6914304	best: 0.6914304 (5)	total: 3.29s	remaining: 27m 21s
6:	test: 0.6919585	best: 0.6919585 (6)	total: 3.73s	remaining: 26m 35s
7:	test: 0.6923066	best: 0.6923066 (7)	total: 4.25s	remaining: 26m 28s
8:	test: 0.6932154	best: 0.6932154 (8)	total: 4.76s	remaining: 26m 23s
9:	test: 0.6936888	best: 0.6936888 (9)	total: 5.28s	remaining: 26m 18s
10:	test: 0.6939511	best: 0.6939511 (10)	total: 5.8s	remaining: 26m 17s
11:	test: 0.6944661	best: 0.6944661 (11)	total: 6.32s	remaining: 26m 13s
12:	test: 0.6946432	best: 0.6946432 (12)	total: 6.78s	remaining: 25m 58s
13:	test: 0.6950020	best: 0.6950020 (13)	t

In [34]:
model = CatBoostClassifier(
    iterations=3000,             # many trees + early stopping
    learning_rate=0.03,          # small = stable
    depth=6,                     # tree depth
    l2_leaf_reg=3,               # regularization
    loss_function="Logloss",     # binary classification
    eval_metric="AUC",           # for Kaggle leaderboard
    bootstrap_type="Bayesian",   # better generalization
    random_strength=1.5,         # randomness in splits
    bagging_temperature=1.0,     # more randomness
                   # row sampling
    colsample_bylevel=0.8,       # column sampling
    random_seed=42,
    od_type="Iter",              # early stopping
    od_wait=100,                 # patience
    task_type="CPU"              # or GPU if enabled
)
model.fit(train_pool , eval_set=valid_pool , verbose=1, use_best_model=True)

0:	test: 0.6763081	best: 0.6763081 (0)	total: 519ms	remaining: 25m 57s
1:	test: 0.6836965	best: 0.6836965 (1)	total: 989ms	remaining: 24m 42s
2:	test: 0.6867931	best: 0.6867931 (2)	total: 1.51s	remaining: 25m 10s
3:	test: 0.6877036	best: 0.6877036 (3)	total: 2s	remaining: 24m 54s
4:	test: 0.6893716	best: 0.6893716 (4)	total: 2.49s	remaining: 24m 52s
5:	test: 0.6892033	best: 0.6893716 (4)	total: 2.98s	remaining: 24m 48s
6:	test: 0.6903220	best: 0.6903220 (6)	total: 3.42s	remaining: 24m 21s
7:	test: 0.6904566	best: 0.6904566 (7)	total: 3.95s	remaining: 24m 36s
8:	test: 0.6904565	best: 0.6904566 (7)	total: 4.42s	remaining: 24m 30s
9:	test: 0.6907966	best: 0.6907966 (9)	total: 4.97s	remaining: 24m 44s
10:	test: 0.6911119	best: 0.6911119 (10)	total: 5.47s	remaining: 24m 46s
11:	test: 0.6911520	best: 0.6911520 (11)	total: 6s	remaining: 24m 55s
12:	test: 0.6912327	best: 0.6912327 (12)	total: 6.52s	remaining: 24m 57s
13:	test: 0.6914838	best: 0.6914838 (13)	total: 6.97s	remaining: 24m 46s
14:	

In [35]:
sub_pred = model.predict(test)
sub_pred
submission = pd.DataFrame({
    "id": test.index,
    "diagnosed_diabetes": sub_pred
})
submission.to_csv('try_catboost.csv', index=False)